# SimResults To Wiki Table Utility

In [67]:
import pandas as pd

### Inputs

##### Series data info

In [68]:
series_directory = "PWCSS 2020"
race_directory = "nurb"
simresults_code = "201108-2GF"

##### Race format info

In [69]:
table_keys = ["q", "r1", "r2"]
table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

##### Table output info

In [70]:
qualy_row_0 = """(% border="1" style="width:554px" %)"""
qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""

race_row_0 = """(% border="1" style="width:747px" %)"""
race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""

### Driver and Points Info Parse

In [71]:
drivers_table_file = "{}/drivers_table.csv".format(series_directory)
points_table_file = "{}/points_table.csv".format(series_directory)

results_file = "{}/{}/{}.csv".format(series_directory, race_directory, simresults_code)
output_file = "{}/{}/wiki_tables.txt".format(series_directory, race_directory)

simresults_url = "https://simresults.net/{}".format(simresults_code)

In [72]:
drivers_table = pd.read_csv(drivers_table_file, dtype=object)

In [73]:
points_table = pd.read_csv(points_table_file, dtype=object)

### Results Tables Parse

In [74]:
names = dict(zip(table_keys, table_names))

rows = {key: [0,0] for key in table_keys}

with open(results_file) as fp:
    current_table = ""
    for i, row in enumerate(fp):
        if current_table:
            if row=="\n":
                rows[current_table][1] = i-1 # manual adjustment
                print("table ended line {}".format(rows[current_table][1]), end="\n\n")
                current_table = ""
            elif i>=rows[current_table][0]:
                print(row, end="")
        else:
            for key in table_keys:
                if row.startswith(names[key]):
                    current_table = key
                    rows[current_table][0] = i+2 # manual adjustment
                    print("found table {} starting line {}".format(names[key], rows[current_table][0]))

# MANUAL ADJUSTMENT :/
# rows["q2"] = [i-1 for i in rows["q2"]]
# rows["r2"] = [i-1 for i in rows["r2"]]

found table Qualify result starting line 15
Pos,   Class, Team, Vehicle, Driver, Laps, Best lap, Gap
"1",   "", "", "ks_porsche_cayman_gt4_clubsport", "PhotonBurst", "6", "01:32.9400", "-",
"2",   "", "", "ks_porsche_cayman_gt4_clubsport", "McLarenTim", "6", "01:33.2540", "00:00.3140",
"3",   "", "", "ks_porsche_cayman_gt4_clubsport", "Waffles", "6", "01:34.0250", "00:01.0850",
"4",   "", "", "ks_porsche_cayman_gt4_clubsport", "NotJames", "6", "01:34.0270", "00:01.0870",
"5",   "", "", "ks_porsche_cayman_gt4_clubsport", "gunciz", "6", "01:34.0340", "00:01.0940",
"6",   "", "", "ks_porsche_cayman_gt4_clubsport", "MrGrinder", "5", "01:34.0930", "00:01.1530",
"7",   "", "", "ks_porsche_cayman_gt4_clubsport", "Trempale", "6", "01:34.2710", "00:01.3310",
"8",   "", "", "ks_porsche_cayman_gt4_clubsport", "Electro", "6", "01:34.3650", "00:01.4250",
"9",   "", "", "ks_porsche_cayman_gt4_clubsport", "Georgin", "6", "01:34.9560", "00:02.0160",
"10",   "", "", "ks_porsche_cayman_gt4_clubsport", "

In [75]:
tables = {}

for key in table_keys:
    table_skiprows = rows[key][0]
    table_nrows = rows[key][1] - table_skiprows
    table_df = pd.read_csv(results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
    table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
    table_df.columns = table_df.columns.str.strip(' \'"')
    table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)
    
    if key.startswith("r"):
        table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "DNF")
    
    print(names[key])
    display(table_df)
    tables[key] = table_df
    

Qualify result


,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,6,01:32.940,-
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,6,01:33.254,00:00.314
2,3,,,ks_porsche_cayman_gt4_clubsport,Waffles,6,01:34.025,00:01.085
3,4,,,ks_porsche_cayman_gt4_clubsport,NotJames,6,01:34.027,00:01.087
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,6,01:34.034,00:01.094
5,6,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,5,01:34.093,00:01.153
6,7,,,ks_porsche_cayman_gt4_clubsport,Trempale,6,01:34.271,00:01.331
7,8,,,ks_porsche_cayman_gt4_clubsport,Electro,6,01:34.365,00:01.425
8,9,,,ks_porsche_cayman_gt4_clubsport,Georgin,6,01:34.956,00:02.016
9,10,,,ks_porsche_cayman_gt4_clubsport,Salzeder,5,01:35.049,00:02.109


Race 1 result


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,14,21:55.144,01:33.355,99.83%,13,0,
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,14,+00:03.072,01:33.165,99.53%,0,0,
2,3,,,ks_porsche_cayman_gt4_clubsport,NotJames,14,+00:20.090,01:34.466,99.59%,0,0,
3,4,,,ks_porsche_cayman_gt4_clubsport,Trempale,14,+00:20.788,01:34.087,99.14%,0,0,
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,14,+00:27.087,01:34.372,98.88%,0,0,
5,6,,,ks_porsche_cayman_gt4_clubsport,Electro,14,+00:27.686,01:34.296,99.27%,0,0,
6,7,,,ks_porsche_cayman_gt4_clubsport,Waffles,14,+00:29.531,01:34.336,98.59%,0,0,
7,8,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,14,+00:29.764,01:34.358,99.19%,0,0,
8,9,,,ks_porsche_cayman_gt4_clubsport,gman197002,14,+00:40.950,01:35.638,99.39%,0,0,
9,10,,,ks_porsche_cayman_gt4_clubsport,Echo51,14,+00:48.079,01:35.393,98.83%,0,0,


Race 2 result


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,14,22:02.873,01:33.322,99.19%,13,0,
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,14,+00:11.239,01:33.377,98.83%,0,0,
2,3,,,ks_porsche_cayman_gt4_clubsport,NotJames,14,+00:14.055,01:34.476,99.49%,0,0,
3,4,,,ks_porsche_cayman_gt4_clubsport,Electro,14,+00:14.822,01:34.265,99.28%,0,0,
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,14,+00:16.595,01:34.433,99.18%,0,0,
5,6,,,ks_porsche_cayman_gt4_clubsport,Trempale,14,+00:17.001,01:33.888,98.68%,0,0,
6,7,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,14,+00:20.302,01:34.281,99.08%,0,0,
7,8,,,ks_porsche_cayman_gt4_clubsport,Waffles,14,+00:24.939,01:34.383,98.61%,0,0,
8,9,,,ks_porsche_cayman_gt4_clubsport,Georgin,14,+00:27.197,01:35.243,99.45%,0,0,
9,10,,,ks_porsche_cayman_gt4_clubsport,Salzeder,14,+00:39.574,01:35.127,98.67%,0,0,


### Wiki Table Printing

In [76]:
def make_qualy_table(table_key):
    
    table_name, table_df = names[key], tables[key]
    
    lines_buffer = [table_name, qualy_row_0, qualy_row_1]
    
    for i, df_row in table_df.iterrows():
        
        if int(df_row["Laps"])==0:
            continue
        
        driver = df_row["Driver"]

        driver_info = drivers_table.loc[drivers_table["ign"]==driver]
        assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)
        
        position = df_row["Pos"]
        number = driver_info["number"].item()
        flag = driver_info["flag"].item()
        name = driver_info["name"].item()
        team = driver_info["team"].item()
        time = df_row["Best lap"]
        if int(position)==1:
            time = "**{}**".format(time)
            
        # print(position, number, flag, name, team, time)
        line = qualy_row_2.format(position, number, flag, name, team, time) if i==0 else qualy_row.format(position, number, flag, name, team, time)
        lines_buffer.append(line)
    
    lines_buffer.append(qualy_row_last.format(simresults_url))
    
    print("made {} rows for table: {}".format(len(lines_buffer), table_name))
    return lines_buffer



def make_race_table(table_key, points_table, add_qualy_points=False):
    
    table_name, table_df = names[key], tables[key]
    
    lines_buffer = [table_name, race_row_0, race_row_1]
    
    for i, df_row in table_df.iterrows():
        
        if int(df_row["Laps"])==0:
            continue
        
        driver = df_row["Driver"]

        driver_info = drivers_table.loc[drivers_table["ign"]==driver]
        assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)
        
        position = df_row["Pos"]
        number = driver_info["number"].item()
        flag = driver_info["flag"].item()
        name = driver_info["name"].item()
        team = driver_info["team"].item()
        laps = df_row["Laps"]
        timeorretired = df_row["Time/Retired"]
        
        session_num = table_key[1:]
        qualy_table = tables["q"] if "q" in tables else tables["q" + session_num]
        grid = qualy_table.loc[qualy_table["Driver"]==driver]["Pos"].item()
        points = "0"
        if (int(position)<=15):
            points = points_table.loc[points_table["pos"]==position]["points"].item()
            if add_qualy_points:
                points_int = int(points) + int(points_table.loc[points_table["pos"]==grid]["qualy_points"].item())
                points = str(points_int) + ("^^{}^^".format(grid) if int(grid)<=3 else "")
            
            
        # print(position, number, flag, name, team, laps, timeorretired, grid, points)
        line = race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
        lines_buffer.append(line)
    
    lines_buffer.append(race_row_last.format(simresults_url))
    
    table_df_withtimes = table_df.copy()
    table_df_withtimes["Best lap"] = pd.to_datetime(table_df_withtimes["Best lap"], format="%M:%S.%f", errors='coerce')
    fastest_driver = table_df.loc[table_df_withtimes["Best lap"]==table_df_withtimes["Best lap"].min()].iloc[0]["Driver"]
    fastest_driver_flag = drivers_table.loc[drivers_table["ign"]==fastest_driver].iloc[0]["flag"]
    fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
    lines_buffer.append(race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))
    
    print("made {} rows for table: {}".format(len(lines_buffer), table_name))
    return lines_buffer

In [77]:
tables_lines = {}

for key in table_keys:
    if key.startswith("q"):
        tables_lines[key] = make_qualy_table(key)
    elif key.startswith("r"):
        if "1" in key:
            tables_lines[key] = make_race_table(key, points_table, add_qualy_points=True)
        else:
            tables_lines[key] = make_race_table(key, points_table)

made 17 rows for table: Qualify result
made 18 rows for table: Race 1 result
made 18 rows for table: Race 2 result


In [78]:
with open(output_file, "w+") as fp:
    for key in table_keys:
        if key in tables_lines:
            fp.write("\n".join(tables_lines[key]) + "\n\n")